## [Is the deconvolution layer the same as a convolutional layer](https://arxiv.org/pdf/1609.07009.pdf)



In [ ]:
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('C:/Anaconda3/envs/tensorflow/Lib/site-packages')
import tensorflow as tf

%matplotlib inline

* [pretty table show](https://matplotlib.org/gallery/images_contours_and_fields/image_annotated_heatmap.html#sphx-glr-gallery-images-contours-and-fields-image-annotated-heatmap-py)

In [ ]:
def show(mts):
    
    if(mts is None):
        print('>>> nothing to show  <<<')
    if(type(mts) != list):
        mts = [mts]

    for i, mt in enumerate(mts):
        X = mt.shape[0]
        Y = mt.shape[1]
        fig, ax = plt.subplots(figsize=(Y, X))        
        im = ax.imshow(mt)

        # We want to show all ticks...
        ax.set_xticks(np.arange(Y))
        ax.set_yticks(np.arange(X))
        # ... and label them with the respective list entries
        ax.set_xticklabels([y for y in range(Y)])
        ax.set_yticklabels([x for x in range(X)])

        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")

        # Loop over data dimensions and create text annotations.
        for x in range(X):
            for y in range(Y):
                text = ax.text(y, x, mt[x, y], fontweight='bold',fontsize='x-large',
                               ha="center", va="center", color="w")

        ax.set_title('matrix%d'%(i))
        fig.tight_layout()
    plt.show()

def expand(mt, stride):
    X = mt.shape[0]
    Y = mt.shape[1]
    strideX = stride[0]
    strideY = stride[1]
    ret = np.zeros((X*strideX,Y*strideY))
    for x in range(X):
        for y in range(Y):
            ret[x*strideX, y*strideY] = mt[x,y]
    return ret

def conv2d(data, kernel):
    data = data.astype(np.float64)
    kernel = kernel.astype(np.float64)
    data = data.reshape(1, data.shape[0], data.shape[1], 1)
    kernel = kernel.reshape(kernel.shape[0], kernel.shape[1], 1, 1)
    input = tf.Variable(data)
    filter = tf.Variable(kernel)

    result = None

    op = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='SAME')
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)

        result = sess.run(op)
    return result

def deconv2d(data, kernel, stride):
    strideX = stride[0]
    strideY = stride[1]
    data = data.astype(np.float64)
    kernel = kernel.astype(np.float64)
    data = data.reshape(1, data.shape[0], data.shape[1], 1)
    kernel = kernel.reshape(kernel.shape[0], kernel.shape[1], 1, 1)
    input = tf.Variable(data)
    filter = tf.Variable(kernel)
    
    input_shape = input.get_shape().as_list()
    filter_shape = filter.get_shape().as_list()
    
    output_shape = [input_shape[0], input_shape[1]*strideX, input_shape[2]*strideY, filter_shape[2]]

    result = None

    op = tf.nn.conv2d_transpose(input, filter, 
                                output_shape=output_shape,
                                strides=[1, strideX, strideY, 1], padding='SAME')
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)

        result = op.eval(session=sess)
    return result

def split(kernel, stride):
    X = kernel.shape[0]
    Y = kernel.shape[1]
    strideX = stride[0]
    strideY = stride[1]
    X0 = int(X/strideX)
    Y0 = int(Y/strideY)
    kernels = []
    for s in range(strideX*strideY):
        k = np.zeros((X0,Y0))
        kernels.append(k)
    for x in range(strideX):
        for y in range(strideY):
            for x0 in range(X0):
                for y0 in range(Y0):
                    kernels[x*strideY+y][x0,y0] = kernel[x0*strideX+x, y0*strideY+y]
    return kernels

def concat(datas):
    R = []
    for d in datas:
        d = d.reshape(1,d.shape[0],d.shape[1],1)
        R.append(d)
    R.reverse()
    op = tf.concat(R,3)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        result = op.eval(session=sess)
    return result

def depth_to_space(data, block_size):
    op = tf.depth_to_space(data,block_size)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        result = op.eval(session=sess)
    return result

def periodic_shuffle1(datas, stride):
    ret = concat(datas)
    ret = depth_to_space(ret, stride[0])
    return ret

def periodic_shuffle2(datas, stride):
    strideX = stride[0]
    strideY = stride[1]
    datas.reverse()
    X = datas[0].shape[0]
    Y = datas[0].shape[1]
    ret = np.zeros((X*strideX,Y*strideY))
    for x in range(strideX):
        for y in range(strideY):
            for x0 in range(X):
                for y0 in range(Y):
                    ret[x0*strideX+x, y0*strideY+y] = datas[x*strideY+y][x0,y0]
    return ret

def contains(A,a):
    x=-1
    y=-1
    l = a[0]
    for x0,L in enumerate(A):
        for y0 in range(len(L)-len(l)):
            r = np.isclose(l, L[y0:y0+len(l)])
            if((np.max(r)==True) and (np.min(r)==True)):
                y = y0
                x = x0
                aA = A[x:x+a.shape[0],y:y+a.shape[1]]
                r = np.isclose(aA, a)
                if((np.max(r)==True) and (np.min(r)==True)):
                    return aA
                else:
                    continue
    return None

In [ ]:
# configuration
# input data shape
D = (4,4)
# stride 
S = (3,3)
# kernel shape
K = (3,4)

In [ ]:
# calculation pad
Kmax = max(K)
Smax = max(S)
if(((Kmax/Smax) > 1) and (int(Kmax/Smax) == (Kmax/Smax))):
    padK = ((0,0),(0,0))
else:
    Kmax = int(round(Kmax/Smax+0.5)*Smax)
    padK = ((0,Kmax-K[0]),(0,Kmax-K[1]))

In [ ]:
data = np.random.randint(1,10,size=D)
show(data)

In [ ]:
data2 = expand(data, S)
show(data2)

In [ ]:
kernel = np.random.randint(1,10,size=K)
kernelPad = np.pad(kernel,padK,mode='constant')
show(kernelPad)

In [ ]:
kernels = split(kernelPad,S)
show(kernels)

In [ ]:
#result = conv2d(data2, kernel)
result = conv2d(data2, kernelPad)
result0 = result.reshape(result.shape[1], result.shape[2])
show(result0)

In [ ]:
kernel22 = np.rot90(kernel,2)
data22 = np.pad(data, [0,int(S[1]/2)], mode='constant')
result2 = deconv2d(data22, kernel22, S)
result2 = result2.reshape(result2.shape[1], result2.shape[2])
                
result3 = contains(result2,result0)
if(result3 is None):
    print('deconv2d result is not matched with tf deconv2d')
    show(result2)
else:
    print('deconv2d result is matched with tf deconv2d, somehow!')
    show(result3)

In [ ]:
results = []
for k in kernels:
    result = conv2d(data, k)
    result = result.reshape(result.shape[1], result.shape[2])
    results.append(result)
show(results)

In [ ]:
#result1 = periodic_shuffle2(results, S)
result1 = periodic_shuffle1(results, S)
result1 = result1.reshape(result1.shape[1], result1.shape[2])
show(result1)

In [ ]:
r = np.isclose(result0, result1)
if((np.max(r)==True) and (np.min(r)==True)):
    print('convolution and deconvolution is the same')
else:
    print('convolution and deconvolution is not the same')